In [ ]:
# load the data
import exputils as eu
import numpy as np
import os
from glob import glob
import re


# parameters
file_name = 'racer'

############################################
# fixed feature experiments
# fixed_feature_dirpath = '/scratchlocal/creinke/data/study/perception/experiments/continuous_sf_01/test_iracer_1000spe_updated_xi_learned_features/experiments'
fixed_feature_dirpath = '/media/creinke/Crucial X6/experiments/continuous_sf_01/312_iracer_1000spe_updated_xi'
learned_feature_dirpath = '../../504_iracer_1000spe_updated_xi_learned_features'

feature_learning_base_experiment_id = '020500'
feature_learning_base_n_phases = 10
is_include_feature_learning_in_average_reward_per_phase_plot = True

# (name, directory, exp_dir, is_add_data_from_feature_learning_base_experiment
experiment_list = [
    ('QL', fixed_feature_dirpath, 20500, False),
    ('SFQL', fixed_feature_dirpath, 40600, False),
#     ('SFQL', fixed_feature_dirpath, 140600, False),
    ('SFQL-R', fixed_feature_dirpath, 30507, False),
#     ('SFQL-R', fixed_feature_dirpath, 130507, False),
    ('SFQL-3', learned_feature_dirpath, 330403, True),
    ('SFQL-6', learned_feature_dirpath, 360403, True),
    ('CXi', fixed_feature_dirpath, 60600, False),
    ('CXi-R', learned_feature_dirpath, 600703, False),
#     ('CMF Xi-R', learned_feature_dirpath, 305300, False),    
    ('CXi-3', learned_feature_dirpath, 630503, True),
#     ('CMF Xi phi=6', learned_feature_dirpath, 660603, True),
]


############################################
# load feature learning base data

def load_initial_phases(exp_id, rep_id, single_rep_data):
    if single_rep_data:
        keep_steps_inds = single_rep_data.phase_per_step < feature_learning_base_n_phases
        single_rep_data.phase_per_step = single_rep_data.phase_per_step[keep_steps_inds] 
        single_rep_data.reward_per_step = single_rep_data.reward_per_step[keep_steps_inds] 
        
feature_learning_base_data, _ = eu.data.load_experiment_data(
    experiments_directory=os.path.join(fixed_feature_dirpath, eu.DEFAULT_EXPERIMENTS_DIRECTORY),
    allowed_experiments_id_list=[feature_learning_base_experiment_id],
    pre_allowed_data_filter = [
        'phase_per_step',
        'reward_per_step'
    ],
    on_repetition_data_loaded = [load_initial_phases]
)


############################################
# create experiment description from list used by the load experiments functions

experiment_descriptions = eu.AttrDict()
for idx, short_exp_descr in enumerate(experiment_list):
    
    exp_name, exp_dir, exp_id, _ = short_exp_descr
    
    experiment_descr = eu.AttrDict()
    experiment_descr.id = exp_id
    experiment_descr.name = exp_name
    experiment_descr.order = idx
    experiment_descr.is_load_data = True
    experiment_descr.directory = os.path.join(
        exp_dir, 
        eu.DEFAULT_EXPERIMENTS_DIRECTORY,
        eu.EXPERIMENT_DIRECTORY_TEMPLATE.format(exp_id)
    ) 
    experiment_descr.short_name = exp_name
    experiment_descr.description = ''
    
    experiment_descr.repetition_ids = []
    repetition_directories = glob(os.path.join(experiment_descr.directory, re.sub('\{.*\}', '*', eu.REPETITION_DIRECTORY_TEMPLATE)))
    for rep_directory in np.sort(repetition_directories):
        rep_id = re.findall(r'\d+', os.path.basename(rep_directory))[0]
        experiment_descr.repetition_ids.append(int(rep_id))
    experiment_descr.repetition_ids.sort()

    experiment_descriptions[exp_id] = experiment_descr
    

##########################################
# load data

# only collect data to plot every n'th step, otherwise the plots are too large
plot_every_nth_step = 500
def get_every_nth_step_inds(data):
    
    # create indicies
    inds = np.full(len(data), False)    
    step = 0
    while step < len(data):
        inds[step] = True
        step += plot_every_nth_step
        
    return inds


def is_experiment_with_feature_learning(exp_id):
    
    is_experiment_with_feature_learning = False
    
    for short_exp_descr in experiment_list:
        if exp_id == short_exp_descr[2] and short_exp_descr[3]:
            is_experiment_with_feature_learning = True
            
    return is_experiment_with_feature_learning


def calc_total_reward(exp_id, rep_id, single_rep_data):
    
    if single_rep_data:
        
        if is_experiment_with_feature_learning:
            
            f_learn_total_reward = np.nansum(feature_learning_base_data[feature_learning_base_experiment_id].repetition_data[rep_id].reward_per_step)
            cumulative_reward_per_phase = np.nancumsum(single_rep_data.reward_per_phase)
            
            single_rep_data.total_reward[0] = cumulative_reward_per_phase[37 - feature_learning_base_n_phases] + f_learn_total_reward
            
        else:
            
            cumulative_reward_per_phase = np.nancumsum(single_rep_data.reward_per_phase)
    
            single_rep_data.total_reward[0]  = cumulative_reward_per_phase[37]

        
def cal_average_reward_per_phase_per_step(exp_id, rep_id, single_rep_data):
    
    if single_rep_data:

        n_steps_per_phase = np.sum(single_rep_data.phase_per_step == 0)
        
        if is_experiment_with_feature_learning(exp_id):

            phase_per_step = np.concatenate([
                feature_learning_base_data[feature_learning_base_experiment_id].repetition_data[rep_id].phase_per_step,
                single_rep_data.phase_per_step + feature_learning_base_n_phases
            ])
            
            if is_include_feature_learning_in_average_reward_per_phase_plot:
            
                reward_per_step = np.concatenate([
                    feature_learning_base_data[feature_learning_base_experiment_id].repetition_data[rep_id].reward_per_step,
                    single_rep_data.reward_per_step
                ])

                step = np.arange(len(reward_per_step))
                data = np.nancumsum(reward_per_step) / (step + 1) * n_steps_per_phase  
            else:
                reward_per_step = single_rep_data.reward_per_step

                step = np.arange(len(reward_per_step))
                data = np.concatenate([
                    [0] * feature_learning_base_n_phases * n_steps_per_phase,
                    np.nancumsum(reward_per_step) / (step + 1) * n_steps_per_phase  
                ]) 
                
                step = np.arange(len(data))

        else:
            phase_per_step = single_rep_data.phase_per_step
            reward_per_step = single_rep_data.reward_per_step 
            step = np.arange(len(reward_per_step))
            data = np.nancumsum(reward_per_step) / (step + 1) * n_steps_per_phase                    

        selection_inds = get_every_nth_step_inds(data)
        single_rep_data.average_reward_per_phase_per_step = data[selection_inds]
        single_rep_data.average_reward_per_phase_per_step_steps = step[selection_inds]
        single_rep_data.average_reward_per_phase_per_step_phase_per_step = phase_per_step[selection_inds]

        
experiment_data, experiment_descriptions = eu.data.load_experiment_data(
    experiment_descriptions,
    pre_allowed_data_filter = [
        'step', 
        'step_per_phase',
        'reward_per_step', 
        'reward_per_phase', 
        'phase_per_step',
        'phase_per_episode',
        'total_reward',
    ],

    on_repetition_data_loaded = [
         calc_total_reward,
        cal_average_reward_per_phase_per_step,
    ],

    post_allowed_data_filter = [
        'average_reward_per_phase_per_step',
        'average_reward_per_phase_per_step_steps',
        'average_reward_per_phase_per_step_phase_per_step',
        'total_reward',
    ]
)


# Average Reward per Phase per Step

In [ ]:
# Plotting of ['average_reward_per_phase_per_step'] 
import exputils as eu
from copy import copy
import plotly
from exputils.gui.jupyter.plotly_meanstd_scatter import plotly_meanstd_scatter



#experiment_ids = [e[2] for e in experiment_list[:-2]]


# default print properties
multiplier = 2

pixel_cm_ration = 36.5

width_full = int(13.95 * pixel_cm_ration) * multiplier
width_half = int(13.95/2 * pixel_cm_ration) * multiplier
width_third = int(13.95/3 * pixel_cm_ration) * multiplier
width_two_third = int(16* 2./3. * pixel_cm_ration) * multiplier

height_default_1 = int(3.4 * pixel_cm_ration) * multiplier

# margins in pixel
top_margin = 0 * multiplier 
left_margin = 35 * multiplier 
right_margin = 0 * multiplier 
bottom_margin = 25 * multiplier 

font_size = 8 * multiplier 
font_family='Times New Roman'

line_width = 1 * multiplier 

default_plot_config =  eu.AttrDict(
    plotly_format = 'svg',
    layout = eu.AttrDict(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        
        xaxis = eu.AttrDict(
            title = 'Tasks Trained',
            showline = True,
            linewidth = 1,
            zeroline=False,
            linecolor='black',
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGrey',
            mirror=True,
        ),
        yaxis = eu.AttrDict(
            title = 'Average Return',
            showline = True,
            linewidth = 1,
            zeroline=False,
            linecolor='black',
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGrey',
            mirror=True,
        ),
        font = eu.AttrDict(
            family=font_family, 
            size=font_size,
            color='black',
            ),
        updatemenus=[],
        width=width_two_third, # in cm
        height=height_default_1, # in cm
        
        margin = eu.AttrDict(
            l=left_margin, #left margin in pixel
            r=right_margin, #right margin in pixel
            b=bottom_margin, #bottom margin in pixel
            t=top_margin,  #top margin in pixel
            ),

        showlegend=True,
        legend=eu.AttrDict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
            
            ), 
        ),
    
#     default_colors = [
#         'rgb(0,158,115)', # green
        
#         'rgb(0,0,0)', # black
#         'rgb(0,0,0)', # black
#         'rgb(0,0,0)', # black
#         'rgb(0,0,0)', # black
        
#         'rgb(213,94,0)',  # orange
#         'rgb(213,94,0)',  # orange
#         'rgb(213,94,0)',  # orange
#         'rgb(213,94,0)',  # orange
#     ],
    
    
    default_colors = [
        'rgb(0,0,0)',      # black
        
        'rgb(0,114,178)',  # dark blue
        'rgb(86,180,233)', # light blue
        'rgb(0,158,115)', # green
        'rgb(0,158,115)', # green
        
        'rgb(213,94,0)',  # orange
        'rgb(213,94,0)', # vermillion
        'rgb(240,228,66)', # yellow
        'rgb(240,228,66)', # yellow
    ],    
    
    
    
#                       'rgb(0,0,0)',      # black
#                       'rgb(86,180,233)', # light blue
#                       'rgb(0,158,115)', # green
#                       'rgb(240,228,66)', # yellow
#                       'rgb(0,114,178)',  # dark blue
#                       'rgb(213,94,0)',  # orange
#                       'rgb(204,121,167)', # pink
#                       'rgb(213,94,0)', # vermillion
    

    
    default_mean_trace = dict(line=dict(width = line_width)),
    
    mean_traces = [
        dict(line = dict(dash = 'solid')),
        
        dict(line = dict(dash = 'solid')),
        dict(line = dict(dash = 'dash')),
        dict(line = dict(dash = 'dot')),
        dict(line = dict(dash = 'dashdot')),
        
        dict(line = dict(dash = 'solid')),
        dict(line = dict(dash = 'dash')),
        dict(line = dict(dash = 'dot')),
        dict(line = dict(dash = 'dashdot')),
    ],
    
            data_filter=eu.AttrDict(
            every_nth_step=eu.AttrDict(
                step=10,
                include_final_step=False),
        ),
   
)

# identify phase of the plotted steps
average_reward_per_phase_per_step_phase_per_step = experiment_data[list(experiment_data.keys())[0]].repetition_data[0]['average_reward_per_phase_per_step_phase_per_step']
average_reward_per_phase_per_step_steps = experiment_data[list(experiment_data.keys())[0]].repetition_data[0]['average_reward_per_phase_per_step_steps']

# identify the middle phase occurence
tickvals = []
ticktext = []

show = False
for phase in np.unique(average_reward_per_phase_per_step_phase_per_step):

    if (phase + 1) % 5 == 0:
   
        n_plotted_steps_per_phase = np.sum(average_reward_per_phase_per_step_phase_per_step == phase)
        middle_step = int(n_plotted_steps_per_phase/2)

        first_occurence_idx = np.where(average_reward_per_phase_per_step_phase_per_step == phase)[0][0]

        tickvals.append(np.arange(len(average_reward_per_phase_per_step_phase_per_step))[first_occurence_idx + middle_step])
        ticktext.append(str(phase + 1))
        show = False

plot_config = copy(default_plot_config)
plot_config.moving_average = eu.AttrDict(n=1)
plot_config.layout.xaxis.tickmode = 'array'
plot_config.layout.xaxis.tickvals = tickvals
plot_config.layout.xaxis.ticktext = ticktext
plot_config.layout.xaxis.range=[0, 15199]
plot_config.layout.yaxis.range=[30000, 80000]
plot_config.error_type = 'sem'


# select data
data, data_labels = eu.data.select_experiment_data(
    experiment_data,
    ['average_reward_per_phase_per_step'],
    experiment_descriptions=experiment_descriptions,
)
# data_labels[0][0] = ''
# print(data_labels)

# plot data
fig = plotly_meanstd_scatter(
    data,
    subplots=eu.AttrDict(subplot_titles=['']),
    labels=data_labels,
    config=plot_config)

plotly.io.write_image(fig, '{}_average_return_per_task_per_step.pdf'.format(file_name))
fig

# Total Reward

In [ ]:
# Plotting of ['average_reward_per_phase_per_step'] 
import exputils as eu
from exputils.gui.jupyter import plotly_box
import plotly
from copy import copy


# default print properties
multiplier = 2

pixel_cm_ration = 36.5

width_full = int(16 * pixel_cm_ration) * multiplier
width_half = int(13.95/2 * pixel_cm_ration) * multiplier
width_third = int(13.95/3 * pixel_cm_ration) * multiplier

height_default_1 = int(3.75 * pixel_cm_ration) * multiplier

# margins in pixel
top_margin = 0 * multiplier 
left_margin = 35 * multiplier 
right_margin = 1 * multiplier 
bottom_margin = 10 * multiplier 

font_size = 8 * multiplier 
font_family='Times New Roman'

line_width = 1 * multiplier 

# Config
# import ipywidgets
# import plotly
# import numpy as np
# plotly.offline.init_notebook_mode(connected=True)

default_plot_config =  eu.AttrDict(
    plotly_format = 'svg',
     init_mode='all',
    layout = eu.AttrDict(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        
        xaxis = eu.AttrDict(
            showline = True,
            linewidth = 1,
            zeroline=False,
            linecolor='black',
            showgrid=False,
            gridwidth=1,
            gridcolor='LightGrey',
            mirror=True,
        ),
        yaxis = eu.AttrDict(
            title = 'Total Return',
            showline = True,
            linewidth = 1,
            zeroline=False,
            linecolor='black',
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGrey',
            mirror=True,
        ),
        font = eu.AttrDict(
            family=font_family, 
            size=font_size,
            color='black',
            ),
       
        updatemenus=[],
        width=width_full, # in cm
        height=height_default_1, # in cm
        
        margin = eu.AttrDict(
            l=left_margin, #left margin in pixel
            r=right_margin, #right margin in pixel
            b=bottom_margin, #bottom margin in pixel
            t=top_margin,  #top margin in pixel
            ),

        showlegend=True,
        legend=eu.AttrDict(
#             xanchor='right',
#             yanchor='top',
#             y=1,
#             x=1,    
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
            
            ), 
        ),
    
#     default_colors = [
#         'rgb(0,158,115)', # green
#         'rgb(0,0,0)', # black
#         'rgb(213,94,0)',  # orange
#         'rgb(213,94,0)',  # orange
#         'rgb(213,94,0)',  # orange
#         'rgb(0,114,178)',  # dark blue
#         'rgb(0,0,0)', # black
#         'rgb(213,94,0)',  # orange
#         'rgb(0,114,178)',  # dark blue
#     ], 
    
    default_colors = [
        'rgb(0,0,0)',      # black
        
        'rgb(0,114,178)',  # dark blue
        'rgb(86,180,233)', # light blue
        'rgb(0,158,115)', # green
        'rgb(0,158,115)', # green
        
        'rgb(213,94,0)',  # orange
        'rgb(213,94,0)', # vermillion
        'rgb(240,228,66)', # yellow
        
        'rgb(204,121,167)', # pink
        'rgb(204,121,167)', # pink       
    ], 
    
    
    
    
#                       'rgb(0,0,0)',      # black
#                       'rgb(86,180,233)', # light blue
#                       'rgb(0,158,115)', # green
#                       'rgb(240,228,66)', # yellow
#                       'rgb(0,114,178)',  # dark blue
#                       'rgb(213,94,0)',  # orange
#                       'rgb(204,121,167)', # pink

    
    default_mean_trace = dict(line=dict(width = line_width)),
    
    traces = [
        dict(fillcolor='rgb(255,255,255)',notched=True),
        dict(fillcolor='rgb(255,255,255)'),
        dict(fillcolor='rgb(255,255,255)'),
        dict(fillcolor='rgb(255,255,255)'),
        dict(fillcolor='rgb(75,75,75)'),
        dict(fillcolor='rgb(255,255,255)'),
        dict(fillcolor='rgb(255,255,255)'),
        dict(fillcolor='rgb(255,255,255)'),
        dict(fillcolor='rgb(255,255,255)'),
        dict(fillcolor='rgb(75,75,75)'),],
)

#         dict(fillcolor='rgb(75,75,75)'),
#         dict(fillcolor='rgb(255,255,255)'),

   
plot_config = copy(default_plot_config)
plot_config.layout.xaxis.range = [-0.4, 0.35]


# select data
data, data_labels = eu.data.select_experiment_data(
    experiment_data,
    ['total_reward'],
    experiment_descriptions=experiment_descriptions,
#    experiment_ids=['000000']
)
#     repetition_ids=self.repetition_ids,
#     output_format=self.output_format,

# plot data
fig = plotly_box(
    data,
    subplots=eu.AttrDict(subplot_titles=['']),
   labels=data_labels,
    config=plot_config)


plotly.io.write_image(fig, '{}_total_return.pdf'.format(file_name))
fig

## Statistical Test

In [ ]:
# test
from exputils.gui.jupyter.tabulate_pairwise import tabulate_pairwise

# select data
data, data_labels = eu.data.select_experiment_data(
    experiment_data,
    ['total_reward'],
    experiment_descriptions=experiment_descriptions,
)


def cell_formater(cell_data):
    if cell_data < 0.001:
        return '< 0.001'
    else:
        return '{:.3f}'.format(cell_data)
    

plot_config = eu.AttrDict(
    tabulate=dict(
        tablefmt='latex'),
    cell_format = cell_formater, 
    pairwise_mode='upper_triangle_not_identity',
)

# plot data
fig = tabulate_pairwise(
    data,
    labels=data_labels,
    config=plot_config)

#print(fig.replace('$','$'))
print(fig)
#display(fig.replace('\$','$'))

## Ratios

In [ ]:
# Plotting of ['statistical difference'] 
import exputils as eu
from exputils.gui.jupyter.tabulate_pairwise import tabulate_pairwise


def ratio(d1, d2):
    return np.mean(d1)/np.mean(d2)

plot_config = eu.AttrDict(
    tabulate=dict(
        tablefmt='latex'),
    pairwise_function = ratio,
    pairwise_mode='full_not_identity',
    cell_format='{:.3f}',
)

# plot data
fig = tabulate_pairwise(
    data,
    labels=labels,
    config=plot_config)

print(fig)